In [5]:
#!pip install matplotlib
import numpy as np
import pandas as pd
#from sklearn.cluster import DBSCAN
#from pykalman import KalmanFilter
import matplotlib.pyplot as plt
from scipy.optimize import linear_sum_assignment
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from matplotlib.cm import ScalarMappable
from scipy.spatial.distance import cdist
import time

import sys
sys.path.append('build')  # Add the 'build' directory to the Python path

import dbscan_module  # Import the shared object file

In [6]:
def plot_kalman_update_point(cluster_points, cluster_center):
    # Assuming cluster_points is a 2D numpy array with x and y coordinates
    #print(cluster_points[-5:])
    num_points = cluster_points.shape[0]
    # Generate a range of colors that go from light to dark
    colors = np.linspace(0.3, 1, num_points)  # Adjust 0.3 to 1 for darker shades
    
    # Plot the cluster points with varying darkness
    scatter = plt.scatter(cluster_points[:, 0], cluster_points[:, 1], c=colors, cmap='Greys')
    
    # Plot the cluster center
    plt.scatter(cluster_center[0], cluster_center[1], color='red', marker='x', label='Cluster Center')
    
    # Add a colorbar, which will automatically use the 'Greys' colormap
    cbar = plt.colorbar(scatter)
    cbar.set_label('Point Intensity (later points darker)')
    
    plt.show()

def plot_dbscan_output(df, labels):
    #print("###############################")
    #print(unique_labels)
    global global_counter
    global_counter +=1
    print("DBSCAN counter: ",global_counter)
    fig, axs = plt.subplots(2, 2, figsize=(10, 10))
    axs[0,0].scatter(df["usec_since_Unix_Start"], df["El."], marker='o', c=labels)
    axs[0,1].scatter(df["usec_since_Unix_Start"], df["Az."], marker='o', c=labels)
    axs[1,0].scatter(df["El."], df["Az."], marker='o', c=labels)

    inds = labels > -1
    axs[1,1].scatter(df["El."][inds], df["Az."][inds], marker='o', c=labels[inds])
    # Extract x and y axis limits
    x_limits = axs[1, 1].get_xlim()
    y_limits = axs[1, 1].get_ylim()
    axs[1,1].scatter(df["El."], df["Az."], marker='o', c=labels)
    axs[1,1].set_xlim(x_limits)
    axs[1,1].set_ylim(y_limits)
    plt.show()

# Extract variables
def plot_log(kalman_log):
    times = []
    filter_ids = []
    predicted_xs = []
    predicted_ys = []
    
    for entry in kalman_log:
        times.append(entry['time'])
        filter_ids.append(entry['filter_id'])
        predicted_xs.append(entry['updated_x'])
        predicted_ys.append(entry['updated_y'])
    
    filter_ids = np.array(filter_ids)
    unique_ids = np.unique(filter_ids)
    print("unique tracks: ", unique_ids)
    
    # Create a custom colormap for only the used unique filter IDs
    num_unique = len(unique_ids)  # Number of unique filter IDs
    colormap = plt.get_cmap('tab10', num_unique)  # Generate a colormap for the number of unique IDs
    
    # Normalize the colormap for the unique filter IDs
    norm = mcolors.Normalize(vmin=min(unique_ids), vmax=max(unique_ids))
    
    # Display scatter plot for predicted_ys
    fig, ax1 = plt.subplots(figsize=(10, 5))
    scatter1 = ax1.scatter(times, predicted_ys, c=filter_ids, cmap=colormap, norm=norm, alpha=0.1)
    ax1.set_ylim(-180, 200)
    
    # Create a ScalarMappable for the color bar to use only the available filter IDs
    sm1 = ScalarMappable(cmap=colormap, norm=norm)
    sm1.set_array([])  # Required for ScalarMappable
    
    # Add a colorbar with only the colors used in the plot
    cbar1 = fig.colorbar(sm1, ax=ax1, ticks=unique_ids, label="Track IDs")
    
    # Shade the regions specified by grouped_time_intervals
    for interval in grouped_time_intervals:
        ax1.axvspan(interval[0], interval[1], color='gray', alpha=0.1)
    
    plt.show()
    
    # Display scatter plot for predicted_xs
    fig, ax2 = plt.subplots(figsize=(10, 5))
    scatter2 = ax2.scatter(times, predicted_xs, c=filter_ids, cmap=colormap, norm=norm, alpha=0.1)
    ax2.set_ylim(30, 200)
    
    # Create a ScalarMappable for the second color bar to use only the available filter IDs
    sm2 = ScalarMappable(cmap=colormap, norm=norm)
    sm2.set_array([])
    
    # Add a colorbar with only the colors used in the plot
    cbar2 = fig.colorbar(sm2, ax=ax2, ticks=unique_ids, label="Track IDs")
    
    # Shade the regions specified by grouped_time_intervals
    for interval in grouped_time_intervals:
        ax2.axvspan(interval[0], interval[1], color='gray', alpha=0.1)
    
    plt.tight_layout()
    plt.show()

def PrintInfo(distance_matrix, associations, unassigned_clusters):
    print("#########################################")
    print("distance matrix")
    print(distance_matrix)
    print(f"associations: {associations}")
    print(f"unassigned_clusters: {unassigned_clusters}")
    for c in unassigned_clusters:
        print("Initializing new filter for cluster: ", c)

In [ ]:
"""
This is a program that performs multi-target tracking. 
Initially, clustering (DBSCAN) is performed on a batch of samples to estimate the number of targets. 
A kalman filter is then initialized for each cluster (i.e. target).
New observations are used to either update an existing kalman filter's state or ignored as noise.
Periodically, batches of observations are used to perform clustering to update the number of targets. 
Each cluster is either associated with an existing kalman filter or used to initialize a new kalman filter.
If a kalman filter is not assocaited with a cluster for a specified number of clustering iterations, the filter is destroyed.
"""
global_counter = 0
class KalmanFilter:
    def __init__(self,
                 transition_matrices,
                 observation_matrices,
                 initial_state_mean,
                 initial_state_covariance,
                 transition_covariance,
                 observation_covariance):
        self.F = np.array(transition_matrices)  # State transition matrix
        self.H = np.array(observation_matrices)  # Observation matrix
        self.Q = np.array(transition_covariance)  # Process noise covariance
        self.R = np.array(observation_covariance)  # Measurement noise covariance
        self.x = np.array(initial_state_mean)  # Initial state estimate
        self.P = np.array(initial_state_covariance)  # Initial covariance estimate

    def predict(self):
        # Predict the state estimate and covariance
        self.x_prior = self.F @ self.x
        self.P_prior = self.F @ self.P @ self.F.T + self.Q

    def update(self, z):
        # Compute innovation
        y = z - self.H @ self.x_prior
        # Compute innovation covariance
        S = self.H @ self.P_prior @ self.H.T + self.R
        # Compute Kalman gain
        K = self.P_prior @ self.H.T @ np.linalg.inv(S)
        # Update state estimate
        self.x = self.x_prior + K @ y
        # Update covariance estimate
        self.P = (np.eye(self.P_prior.shape[0]) - K @ self.H) @ self.P_prior

    def filter_update(self, filtered_state_mean=None, filtered_state_covariance=None, observation=None):
        # If filtered_state_mean and covariance are provided, update self.x and self.P
        if filtered_state_mean is not None:
            self.x = np.array(filtered_state_mean)
        if filtered_state_covariance is not None:
            self.P = np.array(filtered_state_covariance)
        # Predict
        self.predict()
        # Update if observation is provided
        if observation is not None:
            self.update(observation)
        else:
            # If no observation, update x and P with predicted values
            self.x = self.x_prior
            self.P = self.P_prior
        return self.x, self.P

class Tracker:
    def __init__(self, eps=3, min_samples=15, missed_update_threshold=4):
        # DBSCAN parameters
        self.eps = eps  # Max distance between two points to be considered as in the same neighborhood
        self.min_samples = min_samples  # Minimum number of points to form a cluster

        # Initialize storage for Kalman filters and logs
        self.kalman_filters = []
        self.cluster_assignments = []  # To track which Kalman filter corresponds to which cluster
        self.kalman_log = []  # To log the outputs of the Kalman filter

        self.global_counter = 0

        self.missed_update_threshold = missed_update_threshold  # Number of iterations a filter can go without being updated before removal
        self.missed_updates = []  # Track how many iterations each filter has missed being associated with a cluster
        self.next_label = 0

    # Function to initialize a Kalman filter given the initial position
    def initialize_kalman_filter(self, initial_state):
        # State transition matrix: assuming constant velocity model
        transition_matrix = np.array([
            [1, 0, 1, 0],  # X -> X + Vx
            [0, 1, 0, 1],  # Y -> Y + Vy
            [0, 0, 1, 0],  # Vx -> Vx
            [0, 0, 0, 1]   # Vy -> Vy
        ])

        # Observation matrix: we can only observe position, not velocity
        observation_matrix = np.array([
            [1, 0, 0, 0],
            [0, 1, 0, 0]
        ])

        # Initial guess of the state covariance (uncertainty in initial position and velocity)
        initial_state_covariance = np.eye(4) * 1000

        # Covariance of the process noise (uncertainty in the motion model)
        process_covariance = np.eye(4) * 0.01

        # Covariance of the observation noise (uncertainty in the measurements)
        observation_covariance = np.eye(2) * 10

        # Initial state vector [X, Y, Vx, Vy]
        initial_state_mean = np.array([initial_state[0], initial_state[1], 0, 0])  # Assuming zero initial velocity

        # Create the Kalman filter
        kf = KalmanFilter(
            transition_matrices=transition_matrix,
            observation_matrices=observation_matrix,
            initial_state_mean=initial_state_mean,
            initial_state_covariance=initial_state_covariance,
            transition_covariance=process_covariance,
            observation_covariance=observation_covariance
        )

        return kf

    # Function to run DBSCAN
    def run_dbscan(self, df):
        self.global_counter += 1
        X = df[["El.", "Az."]].values.astype(np.float32)
        #db = DBSCAN(eps=self.eps, min_samples=self.min_samples).fit(X)
        #labels = db.labels_
        db = dbscan_module.dbscan(X, self.eps, self.min_samples)
        labels = dbscan_module.label(db, len(X))
        labels = np.array(labels) - 1
        #print(labels)
        #adsfadsf
        ##################################
        plot_dbscan_output(df, labels)
        ##################################

        # Extract cluster end regions
        cluster_centers = self.get_cluster_centroids(X, labels)

        return cluster_centers

    # Function to find optimal association
    def find_optimal_association(self, distance_matrix, threshold):
        num_kalman_filters, num_new_clusters = distance_matrix.shape

        # Step 1: Ignore new clusters where all distances to prior clusters are greater than the threshold
        unassociated_new_clusters = []
        associated_new_clusters = []
        for new_cluster in range(num_new_clusters):
            if np.all(distance_matrix[:, new_cluster] > threshold):
                unassociated_new_clusters.append(new_cluster)
            else:
                associated_new_clusters.append(new_cluster)

        # Step 2: Assign new clusters to prior clusters based on smallest valid distance
        associations = [-1] * num_kalman_filters  # Initialize associations with -1 (no association)
        assigned_new_clusters = set()  # To track already assigned new clusters

        for klm_ind in range(num_kalman_filters):
            min_distance = float('inf')
            best_new_cluster = -1
            for new_cluster in associated_new_clusters:
                if distance_matrix[klm_ind, new_cluster] < threshold and new_cluster not in assigned_new_clusters:
                    if distance_matrix[klm_ind, new_cluster] < min_distance:
                        min_distance = distance_matrix[klm_ind, new_cluster]
                        best_new_cluster = new_cluster
            if best_new_cluster != -1:
                associations[klm_ind] = best_new_cluster
                assigned_new_clusters.add(best_new_cluster)

        return associations, unassociated_new_clusters

    def calculate_distance_matrix(self, cluster_centers):
        distance_matrix = np.zeros((len(self.kalman_filters), len(cluster_centers)))
        if len(self.kalman_filters) != 0 and len(cluster_centers) !=0:
            #predicted_state_means = np.array([kf.predict().x_prior[:2] for kf in self.kalman_filters])
            predicted_state_means = np.array([kf.x_prior[:2] for kf in self.kalman_filters])
            # Calculate the distance matrix using broadcasting
            distance_matrix = np.linalg.norm(predicted_state_means[:, np.newaxis, :] - cluster_centers[np.newaxis, :, :], axis=2)
        return distance_matrix

    def destroy_expired_filters(self):
        # Create a list of indices for filters to keep (those that haven't missed too many updates)
        indices_to_keep = [i for i, count in enumerate(self.missed_updates) if count <= self.missed_update_threshold]

        # Filter kalman_filters, cluster_assignments, and missed_updates using the same indices
        self.kalman_filters = [self.kalman_filters[i] for i in indices_to_keep]
        self.missed_updates = [self.missed_updates[i] for i in indices_to_keep]
        self.cluster_assignments = [self.cluster_assignments[i] for i in indices_to_keep]

    def get_cluster_centroids(self, X, labels):        
        cluster_centers = []
        unique_labels = np.unique(labels)
        for label in unique_labels:
            if label != -1:  # Exclude noise points
                cluster_points = X[labels == label]
                cluster_center = cluster_points[-3:].mean(axis=0)  # Use last 3 points for averaging
                cluster_centers.append(cluster_center)
        return np.array(cluster_centers)

    def increment_missed_counter(self, associated_filters):
        for i in range(len(self.kalman_filters)):
            if i not in associated_filters:
                self.missed_updates[i] += 1

    def initialize_filters_for_clusters(self, unassigned_clusters, cluster_centers):
        for c in unassigned_clusters:
            # Initialize a new Kalman filter for this unassigned cluster center
            new_kf = self.initialize_kalman_filter(cluster_centers[c])
            self.kalman_filters.append(new_kf)

            # Assign a new unique label to this filter
            self.cluster_assignments.append(self.next_label)
            self.next_label += 1

            # Initialize the missed update count for the new filter
            self.missed_updates.append(0)
        
    # Function to update or initialize Kalman filters based on DBSCAN results
    def update_kalman_filters(self, cluster_centers):
        
        #Compute the pairwise distance matrix between Kalman filter predictions and cluster centers
        distance_matrix = self.calculate_distance_matrix(cluster_centers)

        #Solve the optimal assignment using the custom find_optimal_association function
        associations, unassigned_clusters = self.find_optimal_association(distance_matrix, self.eps)

        # To track filters that are associated with a cluster
        associated_filters = set()  
        for r, c in enumerate(associations):
            if c >= 0:  # Check if cluster was assigned
                # Update the Kalman filter with its associated cluster.. may not be necessary
                #kf = self.kalman_filters[r]
                #kf.update(cluster_centers[c])
                associated_filters.add(r)
                self.missed_updates[r] = 0  # Reset the missed count

        self.initialize_filters_for_clusters(unassigned_clusters, cluster_centers)

        PrintInfo(distance_matrix, associations, unassigned_clusters)

        # Increment missed_count for all non-associated filters
        self.increment_missed_counter(associated_filters)

        self.destroy_expired_filters()

    # Function to update the associated Kalman filters from a sample
    def update_kalman_filters_continuous(self, observation, time):
        best_match = None
        best_dist = 10.0  # Adjusted the gating_threshold to be consistent

        # Find the Kalman filter with the closest prediction to the current observation
        for idx, kf in enumerate(self.kalman_filters):
            # Predict the next state with the Kalman filter
            kf.predict()
            predicted_state_mean = kf.H @ kf.x_prior
            #predicted_state_mean = kf.x_prior[:2]
            
            # Calculate the distance from the predicted position to the observation
            dist = np.linalg.norm(predicted_state_mean - observation)

            if dist < best_dist:
                best_dist = dist
                best_match = idx

        # If a valid match is found, update the corresponding Kalman filter
        if best_match is not None:
            kf = self.kalman_filters[best_match]
            kf.update(observation)

            # Log the updated state
            self.kalman_log.append({
                'time': time,
                'filter_id': self.cluster_assignments[best_match],
                'updated_x': kf.x[0],
                'updated_y': kf.x[1]
            })

    # Function to handle batch updates
    def process_batch(self, df_current):
        cluster_centers = self.run_dbscan(df_current)
        if len(cluster_centers) == 0:
            return  # No valid clusters
        self.update_kalman_filters(cluster_centers)


# Load the data as per your existing code
column_names = [
    "usec_since_Unix_Start", "Energy", "El.", "Az.", "TDOA12", "TDOA13", "TDOA14",
    "TDOA23", "TDOA24", "TDOA34", "Xcorr12", "Xcorr13", "Xcorr14",
    "Xcorr23", "Xcorr24", "Xcorr34"
]

# Replace with your actual data path
path_doa = "../../C/deployment_files/2024-8-29-18-37-36-480140_detection"
path_doa = "../../C/deployment_files/2024-9-25-13-5-15-419096_detection"
path_doa = "../../C/deployment_files/2024-10-1-15-19-31-643613_detection"
#path_doa = "../2024-9-12-9-36-14-51349_detection"
df = pd.read_csv(path_doa, delim_whitespace=True, names=column_names, skiprows=1)

# Instantiate the tracker
tracker = Tracker(eps=3, min_samples=15, missed_update_threshold=4)

# Main processing loop (outside the class)
time_intervals = np.linspace(df["usec_since_Unix_Start"].min(), df["usec_since_Unix_Start"].max(), num=20)
grouped_time_intervals = []

for ind in range(len(time_intervals) - 1):
    # Select the data within the current interval
    observations = df[
        (df["usec_since_Unix_Start"] > time_intervals[ind]) &
        (df["usec_since_Unix_Start"] <= time_intervals[ind + 1])
    ]

    # If there are no data points in this interval, skip to the next iteration
    if observations.empty:
        continue

    # Run DBSCAN on even intervals
    if ind % 2 == 0:
        if ind != 0:
            for _, row in observations.iterrows():
                observation = [row["El."], row["Az."]]
                tracker.update_kalman_filters_continuous(observation, row['usec_since_Unix_Start'])

        grouped_time_intervals.append([time_intervals[ind], time_intervals[ind + 1]])
        tracker.process_batch(observations) # Handle batch updates
    else:
        # For odd intervals, handle individual sample updates
        for _, row in observations.iterrows():
            observation = [row["El."], row["Az."]]
            tracker.update_kalman_filters_continuous(observation, row['usec_since_Unix_Start'])

In [ ]:
plot_log(tracker.kalman_log)
print(len(tracker.kalman_filters))
print("hello")